In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
import torch

from gb import simulate

import tsvar

In [122]:
mu = torch.tensor([1.0, 0.9])
beta = torch.tensor([
    [1.0, 1.0],
    [1.0, 1.0],
])
A = torch.tensor([
    [0.1, 0.9],
    [0.0, 0.1]
])
dim = 2

coeffs = torch.cat((mu, beta[0], A.flatten()))

wold_sim = simulate.GrangeBuscaSimulator(mu_rates=mu, Alpha_ba=A, Beta_ba=beta, thinning=True)

In [123]:
events = wold_sim.simulate(5000)
events = [torch.tensor(ev, dtype=torch.float) for ev in events]
end_time = max(50000, max(map(max, events)))
print(f"Simulated {sum(map(len, events))} events with end time: {end_time}")

Simulated 3061 events with end time: 50000


In [129]:
model = tsvar.wold_model.WoldModel()
model.set_data(events, end_time)

C = 1000.0 * torch.ones(len(coeffs))
prior = tsvar.priors.GaussianPrior(dim=None, n_params=len(coeffs), C=C)

optimizer = torch.optim.Adagrad([coeffs], lr=1e-2)

learner = tsvar.learners.MLELearner(
    model=model, 
    prior=prior, 
    optimizer=optimizer,
    tol=1e-5,
    max_iter=100000,
    debug=False)

coeffs0 = torch.tensor(np.random.uniform(0.01, 1.0, size=len(coeffs)), dtype=torch.float)

learner_callback = tsvar.utils.callbacks.LearnerCallbackMLE(coeffs0, coeffs.numpy())

In [130]:
coeffs_hat = learner.fit(events, end_time, coeffs0, callback=learner_callback)

iter: 12930 | dx: 1.01e-04 | loss: 1.27e+04    loss = 12656.43359375 Converged!


In [131]:
coeffs_hat[:dim].detach().numpy().round(2)

array([0.01, 0.  ], dtype=float32)

In [132]:
coeffs_hat[dim:2*dim].detach().numpy().round(2)

array([0.04, 0.14], dtype=float32)

In [133]:
np.reshape(coeffs_hat[2*dim:].detach().numpy(), (2, 2)).round(2)

array([[0.03, 0.01],
       [0.  , 0.08]], dtype=float32)